In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import os
from math import * 
os.getcwd()
print("The current directory is: ")
print(os.getcwd())
path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)
print("Moving up to the parent directory")
print(os.getcwd())

The current directory is: 
c:\Users\nguye\Desktop\Crystal-Plasticity-Journal-Project\notebooks
Moving up to the parent directory
c:\Users\nguye\Desktop\Crystal-Plasticity-Journal-Project


In [3]:
material = "RVE_1_40_D"
CPLaw = "PH"

loadings = ["linear_uniaxial_RD", 
            "linear_uniaxial_TD",
            "nonlinear_biaxial_RD", 
            "nonlinear_biaxial_TD",     
            "nonlinear_planestrain_RD",     
            "nonlinear_planestrain_TD",     
            "nonlinear_uniaxial_RD", 
            "nonlinear_uniaxial_TD"]
# initial_processCurves = np.load(f'results/{material}/{CPLaw}/universal/initial_processCurves.npy', allow_pickle=True).tolist()
# initial_trueCurves = np.load(f'results/{material}/{CPLaw}/universal/initial_trueCurves.npy', allow_pickle=True).tolist()
    

In [7]:
# Example dictionary with double tuples as keys
my_dict = {
    (("dipole", 1), ("omega", 2)): "value1",
    (("dipole", 3), ("omega", 4)): "value2",
    (("dipole", 5), ("omega", 6)): "value3"
}

# Create a new dictionary with modified keys
new_dict = {((("dipmin", key[0][1]), key[1])): value for key, value in my_dict.items() if key[0][0] == "dipole"}

# Print the new dictionary
print(new_dict)

{(('dipmin', 1), ('omega', 2)): 'value1', (('dipmin', 3), ('omega', 4)): 'value2', (('dipmin', 5), ('omega', 6)): 'value3'}


In [11]:
for loading in loadings:
    initial_processCurves = np.load(f'results/{material}/{CPLaw}/universal/{loading}/initial_processCurves.npy', allow_pickle=True).tolist()
    # Create a new dictionary with modified keys
    initial_processCurves_new_dict = {(tuple((("dipmin", key[i][1]) if key[i][0] == "dipole" else key[i] for i in range(len(key))))): value for key, value in initial_processCurves.items()}

    initial_trueCurves = np.load(f'results/{material}/{CPLaw}/universal/{loading}/initial_trueCurves.npy', allow_pickle=True).tolist()
    initial_trueCurves_new_dict = {(tuple((("dipmin", key[i][1]) if key[i][0] == "dipole" else key[i] for i in range(len(key))))): value for key, value in initial_trueCurves.items()}

    np.save(f'results/{material}/{CPLaw}/universal/{loading}/initial_trueCurves.npy', initial_trueCurves_new_dict)
    np.save(f'results/{material}/{CPLaw}/universal/{loading}/initial_processCurves.npy', initial_processCurves_new_dict)

In [5]:
arraylist= []
for loading in loadings:
    initial_processCurves = np.load(f'results/{material}/{CPLaw}/universal/{loading}/initial_processCurves.npy', allow_pickle=True).tolist()
    # Create a new dictionary with modified keys
    #print(initial_processCurves)
    initial_trueCurves = np.load(f'results/{material}/{CPLaw}/universal/{loading}/initial_trueCurves.npy', allow_pickle=True).tolist()
    arraylist.append(list(initial_trueCurves.keys()))
    #print([dict(tuples) for tuples in list(initial_trueCurves.keys())])
# np.save("initial_params", [dict(tuples) for tuples in list(initial_trueCurves.keys())])
if all(element == arraylist[0] for element in arraylist):
    print("All elements are equal")
else:
    print("Not all elements are equal")


All elements are equal


In [16]:
# Your original dictionary
d = {(('tau0', 184.370685), ('a', 4.030016), ('h0', 1919.209605), ('tausat', 664.667834), ('self', 1.453727), ('coplanar', 1.833917), ('collinear', 11.343172), ('orthogonal', 1.497749), ('glissile', 0.867434), ('sessile', 2.774888)): {'strain': np.array([1, 2, 3]), 'stress': np.array([4, 5, 6])},
     (('tau0', 185.370685), ('a', 4.030016), ('h0', 1920.209605), ('tausat', 664.667834), ('self', 1.453727), ('coplanar', 1.833917), ('collinear', 11.343172), ('orthogonal', 1.497749), ('glissile', 0.867434), ('sessile', 2.774888)): {'strain': np.array([1, 2, 3]), 'stress': np.array([4, 5, 6])}
    }

# create new dictionary
new_d = {}
for key, value in d.items():
    new_key = []
    for tup in key:
        if tup[0] in ('tau0', 'h0', 'tausat'):
            new_tup = (tup[0], int(tup[1]*1e6))
        else:
            new_tup = tup
        new_key.append(new_tup)
    new_d[tuple(new_key)] = value

print(new_d)

{(('tau0', 184370685), ('a', 4.030016), ('h0', 1919209605), ('tausat', 664667834), ('self', 1.453727), ('coplanar', 1.833917), ('collinear', 11.343172), ('orthogonal', 1.497749), ('glissile', 0.867434), ('sessile', 2.774888)): {'strain': array([1, 2, 3]), 'stress': array([4, 5, 6])}, (('tau0', 185370685), ('a', 4.030016), ('h0', 1920209605), ('tausat', 664667834), ('self', 1.453727), ('coplanar', 1.833917), ('collinear', 11.343172), ('orthogonal', 1.497749), ('glissile', 0.867434), ('sessile', 2.774888)): {'strain': array([1, 2, 3]), 'stress': array([4, 5, 6])}}


In [24]:
import copy 

for loading in loadings:
    initial_processCurves = np.load(f'results/{material}/{CPLaw}/universal/{loading}/initial_processCurves.npy', allow_pickle=True).tolist()
    initial_trueCurves = np.load(f'results/{material}/{CPLaw}/universal/{loading}/initial_trueCurves.npy', allow_pickle=True).tolist()

    rearrange_trueCurves = {}
    rearrange_processCurves = {}

    # create a new dictionary with stress multiplied by 1e6

    for key, value in initial_processCurves.items():
        new_value = copy.deepcopy(value)
        new_value['stress'] = value['stress'] * 1e6
        rearrange_processCurves[key] = new_value

    for key, value in initial_trueCurves.items():
        new_value = copy.deepcopy(value)
        new_value['stress'] = value['stress'] * 1e6
        rearrange_trueCurves[key] = new_value

    # create new dictionary
    # rearrange_trueCurves = {}
    # rearrange_processCurves = {}
    
    # for key, value in initial_processCurves.items():
    #     new_key = []
    #     for tup in key:
    #         if tup[0] in ('tau0', 'h0', 'tausat'):
    #             new_tup = (tup[0], int(tup[1]*1e6))
    #         else:
    #             new_tup = tup
    #         new_key.append(new_tup)
    #     rearrange_processCurves[tuple(new_key)] = value

    # for key, value in initial_trueCurves.items():
    #     new_key = []
    #     for tup in key:
    #         if tup[0] in ('tau0', 'h0', 'tausat'):
    #             new_tup = (tup[0], int(tup[1]*1e6))
    #         else:
    #             new_tup = tup
    #         new_key.append(new_tup)
    #     rearrange_trueCurves[tuple(new_key)] = value

    #print(rearrange_trueCurves)
    # Create a new dictionary with modified keys
    #print(initial_processCurves)
    #initial_trueCurves = np.load(f'results/{material}/{CPLaw}/universal/{loading}/initial_trueCurves.npy', allow_pickle=True).tolist()
    #print(list(initial_trueCurves.items())[0])
    #print(list(rearrange_processCurves.items())[0])
    #rearrange_processCurves
    #rearrange_processCurves = {(k[2:3] + k[:2] + k[3:]): v for k, v in initial_processCurves.items()}
    #rearrange_trueCurves = {(k[2:3] + k[:2] + k[3:]): v for k, v in initial_trueCurves.items()}
    np.save(f'results/{material}/{CPLaw}/universal/{loading}/initial_trueCurves.npy', rearrange_trueCurves)
    np.save(f'results/{material}/{CPLaw}/universal/{loading}/initial_processCurves.npy', rearrange_processCurves)

In [19]:
my_tuple = (('a', 4.030016), ('h0', 1919.209605), ('tau0', 184.370685), ('tausat', 664.667834), ('self', 1.453727), ('coplanar', 1.833917), ('collinear', 11.343172), ('orthogonal', 1.497749), ('glissile', 0.867434), ('sessile', 2.774888))

new_tuple = (my_tuple[2],) + my_tuple[:2] + my_tuple[3:]

print(new_tuple)

(('tau0', 184.370685), ('a', 4.030016), ('h0', 1919.209605), ('tausat', 664.667834), ('self', 1.453727), ('coplanar', 1.833917), ('collinear', 11.343172), ('orthogonal', 1.497749), ('glissile', 0.867434), ('sessile', 2.774888))
